In [3]:
%run init_script_defines.py

In [4]:
games_trimmed = build_games_trimmed('../Capstone/api/games')
player_id_set = build_player_id_set()
coordinate_shots = build_coordinate_shots()
grouped_data = build_grouped_data()
player_teams = build_player_teams()
team_set = build_team_set()
team_games = build_team_games()
player_id_games = build_player_id_games()
game_shifts = build_game_shifts('../Capstone/api/shifts')
player_pcts_and_groups=build_player_pcts_and_groups(games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_set=team_set, team_games=team_games, player_id_games=player_id_games, coordinate_shots=coordinate_shots, grouped_data=grouped_data)
sadd = build_sadd(games_trimmed=games_trimmed, directory='../Capstone/api/games', game_shifts=game_shifts, player_id_set=player_id_set, player_teams=player_teams, team_set=team_set, team_games=team_games, player_id_games=player_id_games, coordinate_shots=coordinate_shots, grouped_data=grouped_data, player_pcts_and_groups=player_pcts_and_groups)

build_games_trimmed()


[                                                                                   ] Current gamefile: game_2012020069.json[                                                                                   ] Current gamefile: game_2012020070.json[                                                                                   ] Current gamefile: game_2012020071.json[                                                                                   ] Current gamefile: game_2012020072.json[                                                                                   ] Current gamefile: game_2012020073.json[                                                                                   ] Current gamefile: game_2012020074.json[                                                                                   ] Current gamefile: game_2012020075.json[                                                                                   ] Current gamefile: game_2012020076.json[       

[                                                                                   ] Current gamefile: game_2012020137.json[                                                                                   ] Current gamefile: game_2012020138.json[                                                                                   ] Current gamefile: game_2012020139.json[                                                                                   ] Current gamefile: game_2012020140.json[                                                                                   ] Current gamefile: game_2012020141.json[                                                                                   ] Current gamefile: game_2012020142.json[                                                                                   ] Current gamefile: game_2012020143.json[                                                                                   ] Current gamefile: game_2012020144.json[       

[ =                                                                                 ] Current gamefile: game_2012020205.json[ =                                                                                 ] Current gamefile: game_2012020206.json[ =                                                                                 ] Current gamefile: game_2012020207.json[ =                                                                                 ] Current gamefile: game_2012020208.json[ =                                                                                 ] Current gamefile: game_2012020209.json[ =                                                                                 ] Current gamefile: game_2012020210.json[ =                                                                                 ] Current gamefile: game_2012020211.json[ =                                                                                 ] Current gamefile: game_2012020212.json[ =     

[ =                                                                                 ] Current gamefile: game_2012020273.json[ =                                                                                 ] Current gamefile: game_2012020274.json[ =                                                                                 ] Current gamefile: game_2012020275.json[ =                                                                                 ] Current gamefile: game_2012020276.json[ =                                                                                 ] Current gamefile: game_2012020277.json[ =                                                                                 ] Current gamefile: game_2012020278.json[ =                                                                                 ] Current gamefile: game_2012020279.json[ =                                                                                 ] Current gamefile: game_2012020280.json[ =     

...

[ ======                                                                            ] Current gamefile: game_2013020232.json[ ======                                                                            ] Current gamefile: game_2013020233.json[ ======                                                                            ] Current gamefile: game_2013020234.json[ ======                                                                            ] Current gamefile: game_2013020235.json[ ======                                                                            ] Current gamefile: game_2013020236.json[ ======                                                                            ] Current gamefile: game_2013020237.json[ ======                                                                            ] Current gamefile: game_2013020238.json[ ======                                                                            ] Current gamefile: game_2013020239.json[ ======

[ ======                                                                            ] Current gamefile: game_2013020300.json[ ======                                                                            ] Current gamefile: game_2013020301.json[ ======                                                                            ] Current gamefile: game_2013020302.json[ ======                                                                            ] Current gamefile: game_2013020303.json[ ======                                                                            ] Current gamefile: game_2013020304.json[ ======                                                                            ] Current gamefile: game_2013020305.json[ ======                                                                            ] Current gamefile: game_2013020306.json[ ======                                                                            ] Current gamefile: game_2013020307.json[ ======

[ =======                                                                           ] Current gamefile: game_2013020368.json[ =======                                                                           ] Current gamefile: game_2013020369.json[ =======                                                                           ] Current gamefile: game_2013020370.json[ =======                                                                           ] Current gamefile: game_2013020371.json[ =======                                                                           ] Current gamefile: game_2013020372.json[ =======                                                                           ] Current gamefile: game_2013020373.json[ =======                                                                           ] Current gamefile: game_2013020374.json[ =======                                                                           ] Current gamefile: game_2013020375.json[ ======

[ =======                                                                           ] Current gamefile: game_2013020436.json[ =======                                                                           ] Current gamefile: game_2013020437.json

KeyboardInterrupt: 

In [ ]:
with open('../sadd_sample/player_id_games.pkl', 'rb') as file:
    player_id_games = pickle.load(file)
with open('../sadd_sample/team_games.pkl', 'rb') as file:
    team_games = pickle.load(file)
with open('../sadd_sample/team_set.pkl', 'rb') as file:
    team_set = pickle.load(file)
with open('../sadd_sample/player_teams.pkl', 'rb') as file:
    player_teams = pickle.load(file)
with open('../sadd_sample/grouped_data.pkl', 'rb') as file:
    grouped_data = pickle.load(file)
with open('../sadd_sample/coordinate_shots.pkl', 'rb') as file:
    coordinate_shots = pickle.load(file)
with open('../sadd_sample/player_id_set.pkl', 'rb') as file:
    player_id_set = pickle.load(file)
with open('../sadd_sample/game_shifts.pkl', 'rb') as file:
    player_shifts = pickle.load(file)
with open('../sadd_sample/games_trimmed.pkl', 'rb') as file:
    games_trimmed = pickle.load(file)

In [ ]:
with open('../sadd_sample/game_shifts.pkl', 'rb') as file:
    game_shifts = pickle.load(file)

In [ ]:
sadd[8475799]

In [ ]:
player_teams

In [ ]:
get_opposing_players(8470600, [8474628, 8480802, 8470600, 8480848], 2022, player_teams=player_teams, games_trimmed=games_trimmed, player_id_set=player_id_set)

In [ ]:
sadd.get(8475799).get('sadd')

In [ ]:
max([sadd[player].get('sadd') for player in sadd if sadd[player].get('sadd_events', 0) >= 100])

In [ ]:
maxSaddMin100 = max([sadd[player].get('sadd') for player in sadd if sadd[player].get('sadd_events', 0) >= 100])
for a in sadd.items():
    if (a[1]['sadd']) == maxSaddMin100:
        print(a[0])

In [ ]:
sorted([player for player in sadd.items() if player[1]['sadd_events'] >= 100], key=lambda x: x[1]['sadd'], reverse=True)

In [ ]:
type(sorted([player for player in sadd.items() if player[1]['sadd_events'] >= 100], key=lambda x: x[1]['sadd'], reverse=True))

In [ ]:
type(sorted([player for player in sadd.items() if player[1]['sadd_events'] >= 100], key=lambda x: x[1]['sadd'], reverse=True)[0])

In [ ]:
sorted([player for player in sadd.items() if player[1]['sadd_events'] >= 100], key=lambda x: x[1]['sadd'], reverse=True)[0][0]

In [ ]:
playersBySADD = []
for i in sorted([player for player in sadd.items() if player[1]['sadd_events'] >= 100], key=lambda x: x[1]['sadd'], reverse=True):
    playersBySADD.append((i[0],i[1]['sadd']))
print(playersBySADD)